In [11]:
#first job, defines functions to get financial data on companies
#import packages
from datetime import datetime
import lxml
from lxml import html, etree
import requests
import numpy as np
import pandas as pd
import time
from time import sleep
import string
import yfinance as yf
from bs4 import BeautifulSoup
import urllib
import io
import os
import pandasql
from pandasql import sqldf
import re

In [12]:
# let's first make a function that will make the process of building a url easy.
def make_url(base_url , comp):
    
    url = base_url
    
    # add each component to the base url
    for r in comp:
        url = '{}/{}'.format(url, r)
        
    return url

# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    return(np.unique(x)) 

In [13]:
#returns a list of 10-k urls and writes files to csv  (summary files)
def pull_10k_urls():

# START 10K PULL
    start = time.time()
    print("Starting to Pull All 10K URLs ")

    list_10k = []

    year_list = [ 
        '2011','2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', 
        '2020']
    for y in year_list: 
        
        try: 
            print("Pulling Year: "+y)
            #get all 2019 10ks into dataframe of company, date, balance sheet, income statement, cash flow statement

            # PULL DAILY INDEX FILINGS 
            base_url = r"https://www.sec.gov/Archives/edgar/daily-index"

            # The daily-index filings, require a year and content type (html, json, or xml).
            year_url = make_url(base_url, [y, 'index.json'])

            # Display the new Year URL
            # print('-'*100)
            print('Building the URL for Year: {}'.format(y))
            print("URL Link: " + year_url)

            # request the content for 2019, remember that a JSON strucutre will be sent back so we need to decode it.
            content = requests.get(year_url)
            decoded_content = content.json()

            # def get_10k():
            for item in decoded_content['directory']['item']:

                # get the name of the folder
                print('-'*100)
                print('Pulling url for Quarter: {}'.format(item['name']))

                # The daily-index filings, require a year, a quarter and a content type (html, json, or xml).
                qtr_url = make_url(base_url, [y, item['name'], 'index.json'])

                # print out the url.
                print("URL Link: " + qtr_url)

                # Request, the new url and again it will be a JSON structure.
                file_content = requests.get(qtr_url)
                sleep(0.2)
                decoded_content = file_content.json()

#                 print('-'*100)
#                 print('Pulling files')

                #get just the master file urls into a df 
                master_url_list = []
                for file in decoded_content['directory']['item']:
                    if "master" in file['name']:
                        file_url = make_url(base_url, [y, item['name'], file['name']]).replace(".gz","")

                        master_url_list.append(file_url)
                        #print("File URL Link: " + file_url)
                        #sleep(2)
                    else:
                        pass
                #print(master_url_df) 

                #traverse the master url files 
                for u in master_url_list: 
                    #get content of the file 
                    #print("Requesting: " + u)
                    content = requests.get(u).content
                    sleep(0.2)  

                    try: 
                        data = content.decode("utf-8").split('  ')
                        # We need to remove the headers, so look for the end of the header and grab it's index
                        for index, item in enumerate(data):
                            if "ftp://ftp.sec.gov/edgar/" in item:
                                start_ind = index
                        data_format = data[start_ind + 1:]
                        #list to store master data info 
                        master_data = []
                        # now we need to break the data into sections, this way we can move to the final step of getting each row value.
                        for index, item in enumerate(data_format):

                            # if it's the first index, it won't be even so treat it differently
                            if index == 0:
                                clean_item_data = item.replace('\n','|').split('|')
                                clean_item_data = clean_item_data[8:]
                            else:
                                clean_item_data = item.replace('\n','|').split('|')

                            for index, row in enumerate(clean_item_data):

                                # when you find the text file.
                                if '.txt' in row:

                                    # grab the values that belong to that row. It's 4 values before and one after.
                                    mini_list = clean_item_data[(index - 4): index + 1]

                                    if len(mini_list) != 0:
                                        mini_list[4] = "https://www.sec.gov/Archives/" + mini_list[4]
                                        master_data.append(mini_list)
        #                                 print(mini_list)
                                #print(master_data)

                            master_data = pd.DataFrame.from_records(master_data, columns = ['cik', 'name', 'form','date', 'url'])
                           # print(master_data)
                            df_10k = master_data[master_data['form'] == "10-K"]
                            #print(len(df_10k))
                            if len(df_10k) > 0:
                                list_10k.append(df_10k)
                                #print(df_10k)
                            #print(list_10k)
                    except Exception as ex: 
                        print("url failed: "+ u)
                        pass
        except Exception as ex:
            print ("something broke")

    #End Code
    end = time.time()
    print(end - start)
    print("Finished Pulling 10K URLs")
    df_10k = pd.concat(list_10k)
    df_10k = df_10k.drop_duplicates()
    
    print("Writing 10K links to csv")
    #write 10k links to csv
    df_10k.to_csv("df_10k.csv")

# END 10-K PULL

#START FILING SUMMARY PULL
    sleep(10)
    #read in 10k df 
    df_10k_extract = pd.read_csv("df_10k.csv")
    
    #iterate through 10k urls and return df of urls that lead to filing summary 
    print("Start Pulling Filing Summaries")
    start = time.time()
    summary_url_df = []

    base_url = r"https://www.sec.gov"

    for index, row in df_10k_extract.iterrows():
        try: 
            #print(row)
            doc_url = row['url'].replace('-','').replace('.txt','/index.json')
            print("Requesting: "+doc_url)
            content = requests.get(doc_url).json()
            #rate limit of 10/second
            sleep(0.25)

            for file in content['directory']['item']:
                # Grab the filing summary and create a new url leading to the file so we can download it.
                if file['name'] == 'FilingSummary.xml':
                    xml_summary = base_url + content['directory']['name'] + "/" + file['name']
                    new_row = [row['cik'], row['name'], row['date'],  xml_summary]
                    summary_url_df.append(new_row)
        except Exception as ex:
            print("failed: "+ row['url'])

    summary_url_df = pd.DataFrame.from_records(summary_url_df, columns = ['cik', 'name', 'date', 'xml_summary'])
    #print(summary_url_df['xml_summary'][0])

    #write summary urls to a csv
    print("Writing summary urls to csv")
    summary_url_df.to_csv("summary_urls.csv")
    
    #read in summary urls from csv 
    summary_url_extract = pd.read_csv("summary_urls.csv")

    #End Code
    end = time.time()
    print(end - start)
    print("Finished Pulling Filing Summaries")
    
    return(summary_url_extract)
    

In [14]:
url_df = pull_10k_urls()

Starting to Pull All 10K URLs 
Pulling Year: 2011
Building the URL for Year: 2011
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2011/index.json
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR1
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2011/QTR1/index.json
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR2
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2011/QTR2/index.json
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR3
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2011/QTR3/index.json
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR4
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2011/QTR4/index.json
Pulling Y

----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR2
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2019/QTR2/index.json
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR3
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2019/QTR3/index.json
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR4
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2019/QTR4/index.json
Pulling Year: 2020
Building the URL for Year: 2020
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2020/index.json
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR1
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/index.json
----------------------------------------

Requesting: https://www.sec.gov/Archives/edgar/data/1227654/000114036111011013/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1270400/000119312511042015/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1282266/000119312511042169/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1282637/000119312511041569/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1289848/000095012311016457/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/12978/000119312511041217/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1313024/000104746911001083/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1315257/000119312511041968/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1322791/000119312511042015/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1337675/000117891311000578/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1340041/000095012311015858/index.json
Requesting: 

Requesting: https://www.sec.gov/Archives/edgar/data/1357615/000119312511043559/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1361917/000114420411010393/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1368309/000114420411010402/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1368310/000114420411010396/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1396033/000119312511042899/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1403788/000114420411010407/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1403792/000114420411010382/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1405197/000110465911009371/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1419581/000114420411010513/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1421523/000114420411010391/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1423243/000114420411010397/index.json
Requesting

Requesting: https://www.sec.gov/Archives/edgar/data/26780/000114420411010809/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/310522/000095012311017972/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/313616/000119312511044412/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/313927/000119312511045474/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/314808/000031480811000016/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/316300/000119312511045326/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/33213/000110465911009751/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/350894/000119312511045593/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/351817/000035181711000012/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/353230/000144264311000012/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/36029/000095012311017648/index.json
Requesting: https://www.

...

Requesting: https://www.sec.gov/Archives/edgar/data/882361/000117184320001617/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/884269/000143774920004678/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/894627/000089462720000014/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/908315/000090831520000009/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/96869/000009686920000007/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1228454/000122845420000004/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1257640/000156459020010067/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1263762/000110465920031921/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1275014/000156459020010037/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1289636/000128963620000004/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1301787/000130178720000010/index.json
Requesting: http

Requesting: https://www.sec.gov/Archives/edgar/data/1566052/000105640420002813/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1578533/000102024220000059/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1584492/000102024220000064/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1584831/000165495420002991/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1586883/000102024220000062/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1603669/000102024220000067/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1604083/000105640420002745/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1606909/000160690920000018/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1609065/000156459020012167/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1615219/000161521920000014/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1617959/000105640420002759/index.json
Requesting

Requesting: https://www.sec.gov/Archives/edgar/data/788816/000162828020003884/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/799288/000156459020012223/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/825788/000149315220004475/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1574291/000157429120000008/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1635748/000107997420000144/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1639836/000149315220006086/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1648365/000110465920045193/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1694688/000164033420000762/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/22872/000173112220000362/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/797564/000167479620000011/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/894560/000155116320000019/index.json
Requesting: https

Requesting: https://www.sec.gov/Archives/edgar/data/918545/000091854520000002/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/933972/000157570520000049/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/943535/000149315220006335/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1410428/000110465920048780/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/924515/000165495420004236/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1316517/000121390020010244/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1449349/000149315220007270/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1705012/000149315220007184/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1314196/000121390020010370/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1435375/000173112220000416/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/1451797/000121390020010400/index.json
Requesting: ht

Requesting: https://www.sec.gov/Archives/edgar/data/745543/000074554320000025/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/849401/000143774920015110/index.json
Requesting: https://www.sec.gov/Archives/edgar/data/924805/000165495420007646/index.json
Writing summary urls to csv
7261.348129749298
Finished Pulling Filing Summaries


In [9]:
#ingest summary df and pull reports, write to csv 
def pull_reports(df):
        #iterate through df and get financial data reports in a master list of dictionaries
    print("Starting to pull reports")
    start = time.time()

    master_reports = []

    stmt_list = []

    category_list = []
    for index, row in df.iterrows():
        try: 
            xml = row['xml_summary']
            base_url = row['xml_summary'].replace('FilingSummary.xml', '')
            #print(xml)
            content = requests.get(xml).content
            sleep(0.3)
            soup = BeautifulSoup(content, 'lxml')
            # find the 'myreports' tag because this contains all the individual reports submitted.
            reports = soup.find('myreports') 
            #print(reports) 
        #     # loop through each report in the 'myreports' tag but avoid the last one as this will cause an error.
            for report in reports.find_all('report')[:-1]:

                # let's create a dictionary to store all the different parts we need.
                report_dict = {}
                report_dict['cik'] = row['cik']
                report_dict['name'] = row['name']
                report_dict['date'] = row['date']
                report_dict['name_short'] = report.shortname.text.lower()
                report_dict['name_long'] = report.longname.text

                # position and category isnt always available
                try: 
                    report_dict['position'] = report.position.text
                    report_dict['category'] = report.menucategory.text
                except Exception as ex:
                    report_dict['position'] = ''
                    report_dict['category'] = ''

                report_dict['url'] = base_url + report.htmlfilename.text

                # only add statements accounting for missing categories
                if (report_dict['category'] == "Statements" or report_dict['category'] =='') and "consolidated" in report_dict['name_short']: 

                    # append the dictionary to the master list.
                    master_reports.append(report_dict)

                    stmt_list.append(report_dict['name_short'])
                    category_list.append(report_dict['category'])
                    print(report_dict)
                else: 
                    pass
        except Exception as ex:
            print("failed")

    #         # print the info to the user.
    #         print('-'*100)
    #         print(base_url + report.htmlfilename.text)
    #         print(report.longname.text)
    #         print(report.shortname.text)
    #         print(report.menucategory.text)
    #         print(report.position.text)

    #End Code
    end = time.time()
    print(end - start)
    print("Finished Pulling Reports")
    
    print("Writing reports to csv")
    #write reports to csv
    pd.DataFrame(master_reports).to_csv("master_reports.csv")
    
    #pull from csv
    master_reports_extract = pd.read_csv("master_reports.csv")
    print(master_reports_extract)
    return(master_reports_extract)
    

In [10]:
reports_df = pull_reports(url_df)    

Starting to pull reports
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


KeyboardInterrupt: 

In [ ]:
#remove numbers and turn to float else none
def clean_data_str(string):
    result = re.sub('[^0-9]','', string)
    try:
        result = float(result)
    except Exception as ex:
        result = None 
        pass
    return(result)

In [ ]:
def parse_statement_data(report_df):
    start = time.time()
    # let's assume we want all the statements in a single data set.
    statements_data = []
    
    #store parsed metrics in a list of list, turn to df at the end 
    clean_list = []
    
    print("Parsing statement data")
    # loop through each statement url
    for index, row in report_df.iterrows():
        #print(row)
        # define a dictionary that will store the different parts of the statement.
        statement_data = {}
        statement_data['cik'] = row['cik']
        statement_data['name'] = row['name']
        statement_data['statement_name'] = row['name_short']
        statement_data['statement_url'] = row['url']
        statement_data['date'] = row['date']
        statement_data['headers'] = []
        statement_data['sections'] = []
        statement_data['data'] = []
        
        try:
            # request the statement file content
            #print("requesting statement: " + row['url'])
            content = requests.get(row['url']).content
            sleep(0.3)
            report_soup = BeautifulSoup(content, 'html')

            # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
            for index, row in enumerate(report_soup.table.find_all('tr')):

                # first let's get all the elements.
                cols = row.find_all('td')

                # if it's a regular row and not a section or a table header
                if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
                    reg_row = [ele.text.strip() for ele in cols]
                    statement_data['data'].append(reg_row)

                # if it's a regular row and a section but not a table header
                elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
                    sec_row = cols[0].text.strip()
                    statement_data['sections'].append(sec_row)

                # finally if it's not any of those it must be a header
                elif (len(row.find_all('th')) != 0):            
                    hed_row = [ele.text.strip() for ele in row.find_all('th')]
                    statement_data['headers'].append(hed_row)

                else:            
                    print('We encountered an error.')
        except Exception as ex:
            print("Failed")
            pass

        # append it to the master list.
        statements_data.append(statement_data)
    statement_df = pd.DataFrame(statements_data)
   # print(statement_df)

    for index, row in statement_df.iterrows():
        for i in row['data']:
            try: 
                metric_cik = row['cik']
                metric_company = row['name']
                metric_source = row['statement_name']
                metric_year = row['headers'][1][0][-4:]
                metric_name = i[0]
                metric_value = clean_data_str(i[1])
                
                #append parsed info into a list
                clean_list.append([metric_cik,metric_company,metric_source,metric_year,metric_name,metric_value])
#                 print(metric_cik)
#                 print(metric_company)
#                 print(metric_source)
#                 print(metric_year)
#                 print(metric_name)
#                 print(metric_value)
            except Exception as ex:
                pass
    #print(clean_list)
    clean_df = pd.DataFrame.from_records(clean_list)
    clean_df.columns = ['cik', 'company', 'source', 'year', 'metric', 'value']
    clean_df.to_csv("financial_data.csv")
    #print(clean_df)
    
    print("Finished parsing financial data")
    end = time.time()
    print(end - start)
    return(clean_df)

In [ ]:
parsed_statement_df = parse_statement_data(reports_df)
print(parsed_statement_df)

In [ ]:
pysqldf = lambda q: sqldf(q, globals())

distinct_metrics = pysqldf("SELECT DISTINCT source, metric FROM parsed_statement_df ORDER BY 1,2")
distinct_metrics.to_csv("metrics_list.csv")
print(distinct_metrics)

In [ ]:
# #for each cik, get relevant financial data
# def create_finance_table(x):
    
    
    
#     ic_names = [ 'consolidated statements of comprehensive income',
#                 'consolidated statements of comprehensive income (loss)',
#                 'consolidated statements of comprehensive income (parenthetical)', 
#                 'consolidated statements of comprehensive income (parentheticals)', 
#                 'consolidated statements of comprehensive loss', 
#                 'consolidated statements of earnings', 
#                 'consolidated statements of earnings (parenthetical)', 
#                 'consolidated statements of income', 
#                 'consolidated statements of operations', 
#                 'consolidated statements of operations and comprehensive income', 
#                 'consolidated statements of operations and comprehensive income (loss)', 
#                 'consolidated statements of operations and comprehensive income (parenthetical)', 
#                 'consolidated statements of operations and comprehensive loss']
    
#     bs_names = ['consolidated balance sheet', 
#                 'consolidated balance sheet (parenthetical)'
#                 'consolidated balance sheets', 
#                 'consolidated balance sheets (parenthetical)', 
#                 'consolidated balance sheets parenthetical']
    
#     print(x)

In [ ]:
# create_finance_table(parsed_statement_df)

In [ ]:
# #get balance sheet from reports 
# def get_bs(df):
#     #BALANCE SHEET NAMES 
#     # consolidated balance sheet
#     # consolidated balance sheet (parenthetical)
#     # consolidated balance sheets
#     # consolidated balance sheets (parenthetical)
#     # consolidated balance sheets consolidated balance sheet parenthetical
#     # consolidated balance sheets parenthetical
#     bs_reports = master_reports_extract[master_reports_extract['name_short'].str.contains("consolidated balance sheet")]